<h1><center><font color='Red'><b>Team 12's attempt to help the Child Mind Institute</center></b></font></h1>

<font color='blue'><b> In this notebook, we will be explaining our thought process behind what we did so that we could help the Child Mind Institute. We will also mention the challenges we faced and the potential drawbacks with our approaches. We found it better to present our work through jupyter and markdown instead of an app or dashboard since our work does not involve much of data and output visualization. We did think about visualization and wherever we have some ideas about how we could visualize, we will mention those.</b></font>

*We have used some html to style our markdown. It somehow doesn't show it on Github, but it should be seen when it is opened as a jupyter notebook.*

<font color='purple'><b>The data provided by the Child Mind Institue *consisted* of:<br>
1) a folder with docx files for differnt focus groups. The contents of the docx files were mainly dialogues between Moderators and Parents where moderators asked questions based on surveys filled by the Parents pertaining to how lockdown and Technology are affecting their children.<br>
2) some csv files based on inputs from Prolific Academic.<br>
3) a crisis logger csv file.</b></font>

<font color='purple'><b>We started with the docx files because we found those to contain the maximum unstructured text from which we could draw insights. We started with reading a few documents to understand what we need from them. After thinking hard, we realized the best insights can be drawn from text summarization, but for that, we also need to provide the relevant text from the documents. We realized the dialogues from the Parents will be most useful for the Child Mind Institute to understand the Parents'and their children's Pshycology. Based on our thinking, we realized our task can be broadly broken down into:</b></font><br>

<font color='purple'><b> 1) Extracting relevent text (processing the documents)</b></font><br>
<font color='purple'><b> 2) passing the extracted text to a summarizer function.</b></font>

**The first step was to create the right environment for our tasks, which meant cloning the forking and ML-for-good repository, installing the required packages. We realized the requirement of certain packages as we progressed with out tasks. There are some packages that were not used as well in the final code Below cell shows it all**

In [1]:
#!rm -r ML-for-Good-Hackathon
!git clone https://github.com/morganstanley/ML-for-Good-Hackathon.git
!pip install textract
!pip install rake-nltk
!pip install sentence_transformers
!pip install bert-extractive-summarizer
!pip install python-docx
!pip install vaderSentiment

import pandas as pd
import os
import textract
import re
import gensim
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sentence_transformers import SentenceTransformer
#sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
import tensorflow as tf
import tensorflow_hub
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from summarizer import Summarizer
import docx
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Cloning into 'ML-for-Good-Hackathon'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 82 (delta 19), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (82/82), done.
     |████████████████████████████████| 106 kB 6.9 MB/s 
     |████████████████████████████████| 111 kB 39.0 MB/s 
     |████████████████████████████████| 69 kB 4.5 MB/s 
     |████████████████████████████████| 5.6 MB 21.6 MB/s 
     |████████████████████████████████| 10.1 MB 43.9 MB/s 
     |████████████████████████████████| 32.8 MB 87 kB/s 
     |████████████████████████████████| 103 kB 64.3 MB/s 
     |████████████████████████████████| 1.9 MB 44.5 MB/s 
     |████████████████████████████████| 73 kB 2.6 MB/s 
     |████████████████████████████████| 128 kB 58.7 MB/s 
     |████████████████████████████████| 112 kB 53.1 MB/s 
     |████████████████████████████████| 149 kB 55.2 MB/s 
     |███████████████████████████

     |████████████████████████████████| 1.5 MB 7.1 MB/s 
     |████████████████████████████████| 749 kB 48.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 78 kB 3.3 MB/s 
     |████████████████████████████████| 3.1 MB 11.2 MB/s 
     |████████████████████████████████| 3.3 MB 53.0 MB/s 
     |████████████████████████████████| 1.2 MB 18.4 MB/s 
     |████████████████████████████████| 61 kB 485 kB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 895 kB 22.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=81cc23614c6e35e1398343db25f4daebbde7e31a16366812ba252008dd33

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<h2><center><font color='Blue'><b>Document Preprocessing and extracting relevant text</center></b></font></h2>

<font color='purple'><b>As mentioned already, we realized that the most relevent text is Parents'dialogues. Just extracting the Parents'dialogues involved several steps, like removing certain types of strings, collecting a list of all diaglogues and joining dialogues of individual speakers. <br>
 
Broadly, the steps are <br>
1) Remove irrelevant strings like (silence, [inaudible], next line (\n), extra tabs (\t) etc.) <br>
2) We collect each dialogue from the speakers: Parents, Moderators, Speaker, Adminitrator.<br>
3) We ultimately want only the Parents'dialogues to get more insights on the impact of lockdown and tech on their children.<br>
4) All the dialogues of a Parent are combined for further analysis.<br>
5) The function returns a dictionary with keys as Parents and the values as their combined dialogue from the entire doc. 
</b></font>

<font color='purple'><b>An important observation from the results we saw in doc_preprocessing is that it was not very trivial to get absolutely clean text, so it is possible that some times the resulting text has some not so relevant strings. </b></font>

In [2]:
def doc_preprocessing(path):
    """ 
    Code for preprocessing docx files under focusGroups. The steps involved are:
    1) Remove irrelevant strings like (silence, [inaudible], next line (\n), extra tabs (\t) etc.)
    2) We collect each dialogue from the speakers: Parents, Moderators, Speaker, Adminitrator.
    3) We ultimately want only the Parents'dialogues to get more insights on the impact of lockdown and tech on their children.
    4) All the dialogues of a Parent are combined for further analysis.
    5) The function returns a dictionary with keys as Parents and the values as their combined dialogue from the entire doc.
    """
    # reading the docx file with textract
    text = textract.process(path)
    #setting utf-8 for decoding the text since the file read is in bytes
    text = text.decode('utf-8')
    #removing certain substrings like \n, \t, (silence)
    text =text.replace('\n',"")
    text = text.replace('\t',"")
    text = text.replace('(silence)',"")
    text = text.replace('. ','.')
    #splitting sentences by full stop (.) and question mark (?)
    split_sent=re.split(r'[?.]', text)
    #removing empty strings from the list of sentences
    split_sent = list(filter(('').__ne__, split_sent))
    #remove irrelevant sentences like [inaudible 00:13:35]
    split_sent = [sent for sent in split_sent if not re.search("inaudible",sent)]
    #the next two lines are for collecting all dialogues by each speaker
    dial_characters = ('Administrator','Moderator','Parent','Speaker',' Administrator',' Moderator',' Parent',' Speaker')
    dial_start=[ind for ind,l in enumerate(split_sent) if l.startswith(dial_characters)]
    #next, we combine dialogues for each speaker till another speaker starts a dialogue.
    list_of_dial = []
    for i in range(len(dial_start)-1):
        list_of_dial.append('.'.join(split_sent[dial_start[i]:dial_start[i+1]]))
    #we create a list of only the dialogues of Parents
    list_of_Parent_dial = [sent for sent in list_of_dial if sent.startswith('Parent')]
    # we get the index for each parent present in a session
    num_ind = [sent.split(" ")[1].split(':')[0] for sent in list_of_Parent_dial]
    parent_num = list(set([int(word) for sent in num_ind for word in sent.split() if word.isdigit()]))
    # we combine all that a Parent has said in one dialogue of that Parent
    all_dial_comb = []
    for i in parent_num:
        all_dial_comb.append('.'.join([sent for sent in list_of_Parent_dial if sent.startswith(('Parent '+str(i)+':',' Parent '+str(i)+':'))]))
    # list with slight processing of each Parent dialogue    
    all_d_comb_n=[]
    for i in range(len(all_dial_comb)):
        all_d_comb_n.append(all_dial_comb[i].replace('Parent '+str(i+1)+":","").replace(' Parent '+str(i+1)+":",""))
    # dictionary with keys as Parent names (eg: Parent 1) and the values as their combined dialogue.
    dial_dict = dict(zip(['Parent '+str(i) for i in parent_num],all_d_comb_n))
    # list processing of the values in the dict and return the dict.
    for key in list(dial_dict.keys()):
        dial_dict[key] = dial_dict.get(key).replace("My name's Parent",'').replace('Parent','').replace('Moderator','').replace('Administrator','')
    #dial_dict[key] = re.sub('[0-9]','',dial_dict.get(key))
    return dial_dict

**A quick execution of the doc_preprocessing function above can be seen below. We just chose a single file for demo, we will see later that we automatically do it for all the files and save the output in doc files, in a loop.**

In [3]:
doc_preprocessing('./ML-for-Good-Hackathon/Data/FocusGroups/Media_Group4.docx')

{'Parent 1': "Cool.Oh yes, sorry.Alright, sorry again.My name is  1.I have two children, 10 and 7 year old.Well again I have two children one is 10 and the other one is seven.So 5th grade and 1st grade.So first off my older one didn't really use the computer that often before the pandemic but he likes using technology.So somehow he was happy when it did happen, going remote, and the one thing I like and again my son, the older one has an IEP, he helped me out.So to have a better idea about what was being done at school and how he was doing it.So I was more on top in being a to assist him.The draw back is that sometimes the teachers would make comments and tell him you didn't do this right but there was no follow up because one teacher was teaching remotely, the other one was doing in person.So I would say it did affect him because he was not getting the support he used to.So finally when they were able to go to blended learning he was fine with it but when he, two or three weeks ago, h

<font color='purple'><b> After getting the relevant text from the focus group documents, we needed to get insights from this relevant text. We realized the best solution would be to perform summarization. </b></font><br> 

<font color='purple'><b> There are two types of text summarization in NLP: Abstractive and Extractive. We were excited to try abstractive text summarization which would summarize by understanding the language context. Abstractive summarization uses the variants of the well known Neural Network architecture 'Transformers', but we weren't getting very appropriate results using that.</b></font> <br> 

<font color='purple'><b> Our conclusion is that it didn't work because the language is the documents isn't very clean like how it is in well written articles, but it is in the form of dialogues.</b></font> <br> 

<font color='purple'><b> We finally stuck to extractive text summarization.</b></font> <br> 

<font color='purple'><b> We were also trying key phrase extraction, but it didn't give very meaningful results, we think that it's because the documents don't contain clean language. </b></font> <br> 

<font color='purple'><b> For extractive text summarization, one could either summarization by percentage of text we want from the original text or by word count or in some cases, by number of sentences. We used two extractive text summarizers, one from the Gensim package (with percentage specification and word count specification) and other using the Bert extractive summarizer(with percentage specification and number of sentences specification). We realized some summarizers did better in some cases and others did better in the rest. Therefore, the purpose to use multiple summarizers is that the people from Child Mind Institute can collectively get insights from each type of summarizer. It is also to be noted that in some documents, some parents hardly had sufficient dialogue and the Gensim summarizer particularly gave a warning that there aren't relevant sentences or enough sentences. Below is the function for extractive text summarization.</b></font>

<font color='purple'><b> We start with creating a Bert summarizer object as that loads the Bert model</b></font>

In [4]:
#Creating the Bert Summarizer object
model = Summarizer()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [5]:
def extractive_text_summarization(path):
    """
    This function uses the processed data from doc_preprocessing function. We perform an extractive text summarization for each Parents dialogues.
    We have used the summarizer from gensim package and we also use the Bert summarizer. With these packages, we can extract important text based on word 
    count, percentage, number of sentences. We had tried key phrase extraction which didn't work well.
    """
    #first run the doc_preprocessing function for the file in the path
    imp_dial_parents = doc_preprocessing(path)
    # Summary (0.5% of the original content): Gensim
    summ_per = [summarize(dialogue.replace(".",". "), ratio = 0.1) for dialogue in list(imp_dial_parents.values())]
    # Summary (200 words): Gensim
    summ_words = [summarize(dialogue.replace(".",". "), word_count = 200) for dialogue in list(imp_dial_parents.values())]
    #creating dictionaries with keys as Parents (eg:Parent 1) and values as their summarized dialogues
    keys = list(imp_dial_parents.keys())
    summ_per = dict(zip(keys,summ_per))
    summ_words = dict(zip(keys,summ_words))
    # Using Bert extractive summarizer with ratio specification
    result1 = [model(dialogue, ratio=0.2) for dialogue in list(imp_dial_parents.values())] 
    # Using Bert extractive summarizer with number of sentences specification
    result2 = [model(dialogue, num_sentences=6) for dialogue in list(imp_dial_parents.values())]
    #creating dictionaries with keys as Parents (eg:Parent 1) and values as their summarized dialogues
    bert_ratio = dict(zip(keys,result1))
    bert_num_sent = dict(zip(keys,result2))

    # Ingnore the below commented code. This was an attempt to extract key phrases, but we didn't get great results. We have still kept it since we tried it.
    # r = Rake()
    # key_ph_list = []
    # for i in range(len(keys)):
    #   r.extract_keywords_from_text(list(imp_dial_parents.values())[i])
    #   key_ph_list.append(r.get_ranked_phrases()[:30])
    # key_phrases = dict(zip(keys,key_ph_list))

    #return all the dictionaries with keys as Parents and values as extractive summaries for different types of summarizers
    return summ_per, summ_words, bert_ratio, bert_num_sent

**A quick execution of the extractive_text_summarization function above can be seen below. We just chose a single file for demo, we will see later that we automatically do it for all the files along with the doc_preprocessing function and save the output in doc files, in a loop. Note that the first two summarizations is from Gensim (with word count and ratio/percentage specifications respectively) and the last two are from Bert (with word count and number of sentences specifications respectively). We take care of this detail in the final execution of the loop where we process and perform this function on all doc files.**

In [6]:
extractive_text_summarization('./ML-for-Good-Hackathon/Data/FocusGroups/Media_Group4.docx')

({'Parent 1': "So somehow he was happy when it did happen, going remote, and the one thing I like and again my son, the older one has an IEP, he helped me out.\nAnd what my older one, the 10 year old, does is more like Minecraft and their group chat was just with their friends and again I went from controlling one hour a day to now it's unlimited time.",
  'Parent 2': "I feel initially my daughter went to Sea last year and so initially they were used to doing they had online setup, St Joseph by the Sea, and it was going great.\nAnd blended a Tottenville means you go sit in a classroom with your computer, just like you're at home because all the teachers are at home, and the kids are in school.\nYes. Sea did, Tottenville had to learn how to do it, which I guess they did between last year for them was a total waste because there were kids who just didn't know what they were doing.\nThis year they know what they're doing.\nBut that is something that I think, I don't know if it's  5 had sa

**The below code executes doc_preprocessing and extractive text summarization for all the doc files. We also save the results in doc files.**

In [7]:
list_of_docs = ['./ML-for-Good-Hackathon/Data/FocusGroups/'+file for file in os.listdir('./ML-for-Good-Hackathon/Data/FocusGroups')]



#Executing the extractive_text_summarization function and saving the outputs
para_topics = ['gensim_summ_ratio', 'gensim_summ_words', 'bert_summ_ratio', 'bert_summ_num_sent']
for doc in list_of_docs:
    extract = extractive_text_summarization(doc)
    mydoc = docx.Document()
    for i in range(len(extract)):
        mydoc.add_paragraph(para_topics[i]+':')
        for key in extract[i].keys():
            mydoc.add_paragraph(key+': '+extract[i].get(key))
    mydoc.save('summ_outputs_'+doc.split('/')[-1])

<font color='purple'><b> A possible way to visualize outputs from the summarizer is by creating some kind of an app where we just upload a document and it hightlights the extracted text in the document itself. This could be very beneficial for the Child Mind Institue since they would be looking at the summarized extracted text as part of the document itself and if they want, they could look at the text around the highlighted text to get more context, this is still work in progress.</b></font><br>

<h2><center><font color='Blue'><b>Processing data from Prolific Academic and drawing insights from it</center></b></font></h2>

<font color='purple'><b> When we saw the Prolific Academic csv files, we realized that there are too many columns and we started wondering what insights could we possibly draw from such a file. We started looking at a group of columns and found a feature called "suspectedinfected" in the Nov 2020 and April 2021 files and felt that that could be considered a target feature in some classification type model. We also noticed several columns that had too many missing values and felt that those might not give much insights. We also found a bunch of features with sentences abd started thinking about how we could represent them if we were to create a model. After thinking hard, below are the steps we followed to draw some insights. </b></font><br>

<font color='purple'><b> We followed each of these steps individually for Adult and Parent data from Nov 2020 and April 2021, and did not combine the data because we felt that a more granular analysis might also give more insights. We later received an email about an update in the Prolific academic data, so we further looked at the features to see if there were any changes. We realized that the same approach can be used even on the updated data. Our outputs are finally created for each file even from the updated Nov 2020 and April 2021 data.</b></font><br>

<font color='purple'><b>
1) remove features with more than 50% NaNs.<br>
2) find the top 19 or 20 features that are most correlated to suspectedidentified among each of int and float features.<br>
3) replace NaNs in the string features with 'Information not available'.<br>
4) combine dataframes from 2) and 3).<br>
5) remove all rows with NaNs.<br>
6) set target as suspectedidentified. <br>
7) for the string features, get embeddings for each using the Universal sentence encoder pretrained model from tensorflow hub.<br>
8) For each of the string features, step 7) would give us embeddings of dimension 571. We perform PCA on each to reduce the dimension to 5 for each.<br>
9) In 8), the new features as embeddings are given names as feature_name followed by an index.<br>
10) By step 9), we are done with preprocessing. We then fit a random forest classifier to arrive at the important features. This might help Child Mind
Institute to focus on a fewer features. In case of embeddings, the features for which some components of the embeddings are important can be focussed on.
</b></font><br>

<font color='purple'><b> An important point to note is that our RF classifier is not created here for the purpose of prediction, we mainly want to assess feature importance given the data. Having said that, if we get a data for a quarter with exactly the same set of features as in the any of the csv files above, we can try prediction as well </b></font><br>

<font color='purple'><b> Below is the code demonstrating each step above. It is to be noted that above works when we can set 'suspectedinfected' as the target. We noticed that for the files from quarters or months before Nov 2020, 'suspectedinfected' was not a feature. Depending on how future turns out after Covid, the feature may or may not be there. In those situations, we would again have to look at all features and see if something can be considered a target feature. If we don't find anything that can be used as an appropriate target, we may have to look for different ways to draw insights.</b></font><br>

<font color='purple'><b> We start with loading the Universal Sentence Encoder pretrained model. We set the parameter in the KerasLayer from tensorflow hub as False since we will not be using these embeddings for fine tuning or feature extraction.</b></font><br>

In [8]:
uni_encoder = tensorflow_hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',trainable=False)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


In [9]:
def preprocess_prolificacademic_and_feature_importances(path):
    """
    only works for Nov 2020 and April 2021 data since the data before Nov 2020 doesn't contain the variable suspectedinfected. We set the target
    feature as suspectedidentified and fit a random forest classifier on the processed dataset. Note that the purpose of the classifier is not to
    predict because we realize that the features in the data need not be same always. The purpose is to just suggest the more important features in
    the processed dataset that can. The preprocessing steps are the following:
    1) remove features with more than 50% NaNs.
    2) find the top 19 or 20 features that are most correlated to suspectedidentified among each of int and float features.
    3) replace NaNs in the string features with 'Information not available'.
    4) combine dataframes from 2) and 3).
    5) remove all rows with NaNs.
    6) set target as suspectedidentified. 
    7) for the string features, get embeddings for each using the Universal sentence encoder pretrained model from tensorflow hub.
    8) For each of the string features, step 7) would give us embeddings of dimension 571. We perform PCA on each to reduce the dimension to 5 for each.
    9) In 8), the new features as embeddings are given names as feature_name followed by an index.
    10) By step 9), we are done with preprocessing. We then fit a random forest classifier to arrive at the important features. This might help Child Mind
    Institute to focus on a fewer features. In case of embeddings, the features for which some components of the embeddings are important can be focussed on.
    """
    data2 = pd.read_csv(path)
    #remove features with >50% NaNs and also timestamps and ID columns
    to_be_removed = list((data2.isnull().sum() * 100 / len(data2) >50)[data2.isnull().sum() * 100 / len(data2) >50].index)
    time_col = [col for col in data2.columns if col.startswith('timestamp')]
    to_be_removed = to_be_removed+['ID']+time_col
    data2.drop(to_be_removed, axis=1, inplace=True)
    #finding and grouping features by their dtype. There are some of type int64, some of type float64 and some string features (dtype: object)
    g = data2.columns.to_series().groupby(data2.dtypes).groups
    g_dic = {k.name: v for k, v in g.items()}
    #for all the float features, find the top 20 most correlated with the feature 'suspectedinfected'and create a subset of the dataframe with only these features
    float_f = list(g_dic.get('float64'))
    f_var = data2[float_f]
    f_var_cor = f_var.corr()
    f_var_list = list(f_var_cor.suspectedinfected.sort_values(ascending=False)[1:20].index)
    print(f_var_list)
    f_var = f_var[f_var_list]
    #for all the int features, find the top 20 most correlated with the feature 'suspectedinfected'and create a subset of the dataframe with only these features
    int_f = list(g_dic.get('int64'))
    int_var = data2[int_f+['suspectedinfected']]
    i_var_corr = int_var.corr()
    i_var_list = list(i_var_corr.suspectedinfected.sort_values(ascending=False)[0:20].index)
    print(i_var_list)
    int_var = int_var[i_var_list]
    #for all string features, replace NaN by 'Information not available'
    var_to_preprocess = list(g_dic.get('object'))
    string_f = data2[var_to_preprocess]
    string_f=string_f.fillna('information not available')
    #create a combined dataframe with the 20 float, 20 int and the string features
    combined_df = pd.concat([f_var, int_var,string_f], axis=1)
    print(f"'% of rows usable for training':{len(combined_df.dropna())/len(combined_df)}")
    #drop rows with NaNs. The above line prints the % of rows that remain
    combined_df.dropna(inplace=True)
    #set target as suspectedinfected
    target = combined_df.pop('suspectedinfected')
    print(len(combined_df))
    #for each string feature in combined_df, get embeddings with the pretrained Universal Sentence Encoder model and then perform PCA to reduce dimension of embeddings of each feature to 5
    for feat in var_to_preprocess:
        a=uni_encoder(list(combined_df[feat]))
        principal=PCA(n_components=5)
        principal.fit(a)
        x=principal.transform(a)
        y = pd.DataFrame(x,columns=[feat+str(i) for i in range(1,6)],index=combined_df.index)
        combined_df = pd.concat([combined_df,y],axis=1)
    #drop the original string features    
    combined_df.drop(var_to_preprocess,axis=1,inplace=True)
    #fit a Random Forest Classifier with X=combined_df and Y=target(suspectedinfected)
    clf = RandomForestClassifier(n_estimators=500,max_depth=10,random_state=42)
    clf.fit(combined_df,target)
    #get feature importances as dict and then, convert to a dataframe, return the top 50 important features.
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(combined_df.columns, clf.feature_importances_):
        feats[feature] = importance #add the name/value pair 
    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    importances = importances.sort_values(by='Gini-importance',ascending=False)
    #importances.head(50)
    return importances.head(50)

**A quick execution of the preprocess_prolificacademic_and_feature_importances function above can be seen below. We just chose a single file for demo, we will see later that we automatically do it for all the files in a loop.**

In [10]:
preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/updated_data/November/CRISIS_Adult_November_2020.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (93) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['coviddx', 'priorfatigue_2', 'worriedothers', 'livewithessentialworker', 'friendchangestress', 'priorfatigue', 'essentialworker', 'difficultydistancing', 'emotionsinterfersocial', 'lackcompanionship', 'emotionsinterferrelationships', 'troublesleeping_2', 'priorirritable_2', 'isolated', 'worriedphysical', 'familychangestress', 'priorirritable', 'priornegthoughts_2', 'worriedyourself']
['suspectedinfected', 'symptoms___1', 'symptoms___10', 'symptoms___2', 'symptoms___3', 'symptoms___9', 'symptoms___5', 'symptoms___7', 'symptoms___8', 'impact___3', 'symptoms___4', 'exposed___1', 'exposed___3', 'symptoms___6', 'diagnosedfamily___1', 'impact___1', 'symptoms___12', 'diagnosedfamily___3', 'impact___4', 'exposed___2']
'% of rows usable for training':0.9595141700404858
2133


,Gini-importance
symptoms___1,0.106555
symptoms___10,0.064339
symptoms___2,0.051842
adult_selfreport_baseline_form_timestamp5,0.037943
symptoms___9,0.036935
symptoms___3,0.035173
adult_selfreport_baseline_form_timestamp4,0.032321
adult_selfreport_baseline_form_timestamp1,0.031482
adult_selfreport_baseline_form_timestamp2,0.030652
adult_selfreport_baseline_form_timestamp3,0.030590


**The below code executes preprocess_prolificacademic_and_feature_importances for all the prolific academic csv files (old as well as updated ones from Nov 2020 and April 2021). We also save the results in a single xlsx file with multiple sheets, each containing results from a single input csv file and named accordingly.**

In [11]:
April2021_adult_feat_imp=preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/April 2021/Data/CRISIS_Adult_April_2021.csv')
Nov2020_adult_feat_imp=preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/November 2020/Data/CRISIS_Adult_November_2020.csv')
April2021_parent_feat_imp=preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/April 2021/Data/CRISIS_Parent_April_2021.csv')
Nov2020_parent_feat_imp=preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/November 2020/Data/CRISIS_Parent_November_2020.csv')
updated_April2021_adult_feat_imp=preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/updated_data/April_21/CRISIS_Adult_April_2021.csv')
updated_April2021_parent_feat_imp=preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/updated_data/April_21/CRISIS_Parent_April_2021.csv')
updated_Nov2020_adult_feat_imp = preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/updated_data/November/CRISIS_Adult_November_2020.csv')
updated_Nov2020_parent_feat_imp = preprocess_prolificacademic_and_feature_importances('./ML-for-Good-Hackathon/Data/ProlificAcademic/updated_data/November/CRISIS_Parent_November_2020.csv')

#saving results from preprocess_prolificacademic_and_feature_importances in a single xlsx file
def save_xls(list_dfs, xls_path,required_dfs):
    with pd.ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,required_dfs[n])
        writer.save()


xls_path = '/content/prolificacademic_feature_imp.xlsx'

#getting names of the dataframe created above as strings
required_dfs = [df for df in globals() if 'feat_imp' in df]

#creating the list of dataframes in the same order as their names as string are listed in required_dfs
list_dfs = [April2021_adult_feat_imp,
 Nov2020_adult_feat_imp,
 April2021_parent_feat_imp,
 Nov2020_parent_feat_imp,
 updated_April2021_adult_feat_imp,
 updated_April2021_parent_feat_imp,
 updated_Nov2020_adult_feat_imp,
 updated_Nov2020_parent_feat_imp]

required_dfs = [df[:31] if len(df) > 31  else df for df in required_dfs]

save_xls(list_dfs,xls_path,required_dfs)


<h2><center><font color='Blue'><b>Sentiment Analysis on the crisis logger csv file</center></b></font></h2>

<font color='purple'><b> The crisis logger csv file already looked process and the immediate thought was to perform sentiment analysis. We used Vader to get Positive, neutral and negative scores for each transcription and found the overall rating for each. We finally save the results in a xlsx file along with the other details from the crisis logger file. Below is the function, the execution and how we finally save the result </b></font><br>

In [12]:
def sentiment_scores(sentence):
    """
    getting sentiment scores for every sentence in crisislogger file.
    """
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict=sid_obj.polarity_scores(sentence)
    Negative_score = sentiment_dict['neg']*100
    Neutral_score = sentiment_dict['neu']*100
    Positive_score = sentiment_dict['pos']*100
    if (sentiment_dict['compound'] >= 0.05):
        Overall_rate = 'Positive'
    elif (sentiment_dict['compound'] <= - 0.05):
        Overall_rate = 'Negative'
    else :
        Overall_rate = 'Neutral'
    return Negative_score,Neutral_score,Positive_score,Overall_rate



#executing sentiment_scores on crisislogger data and combining the output with the crisis logger file itself, exporting the result into an xlsx file.
data = pd.read_csv(r"./ML-for-Good-Hackathon/Data/CrisisLogger/crisislogger.csv") 
res = data['transcriptions'].apply(sentiment_scores)
data['Negative_score'] = [x[0] for x in res]
data['Neutral_score'] = [x[1] for x in res]
data['Positive_score'] = [x[2] for x in res]
data['Overall_rate']=[x[3] for x in res]
data.to_excel('./sentiment_scores_crisislogger.xlsx')

In [13]:
data.head(10)

,upload_id,transcriptions,Negative_score,Neutral_score,Positive_score,Overall_rate
0,10,so high our experience so far has been a littl...,8.0,76.4,15.6,Positive
1,209,I'm not going to stay in my name for the anony...,7.9,85.8,6.3,Negative
2,216,so far I have been florentines now for about a...,7.3,85.4,7.3,Neutral
3,222,it has actually been a very difficult. Trying ...,11.1,80.4,8.5,Negative
4,228,so this whole situation has been strange for u...,8.1,78.8,13.1,Positive
5,246,my name is Simi and since the covid virus has ...,9.8,82.5,7.7,Negative
6,342,My fear is that my son will be emotionally imp...,14.6,69.6,15.8,Positive
7,349,Teachers are struggling to support students re...,12.4,77.3,10.3,Negative
8,376,I personally hate Distance Learning. I hate th...,16.1,67.0,17.0,Positive
9,392,I feel anxious often and unsure about my abili...,7.2,79.8,12.9,Positive


<h2><center><font color='Blue'><b> Thank you so much for this amazing opportunity! We are extremely happy that we got to work for such a wonderful cause.</center></b></font></h2>